In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
warnings.filterwarnings('ignore')  # 忽略警告信息

# 自定义交互特征转换器
class InteractionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self  # 无需要拟合的参数
    
    def transform(self, X):
        # 生成区域特征与面积特征的交互项
        if X.shape[1] > 1:
            area_features = X[:, -1].reshape(-1, 1)  # 提取面积特征
            interaction_features = X[:, :-1] * area_features  # 区域特征与面积特征交互
            return np.hstack([X, interaction_features])  # 合并原始特征与交互特征
        return X

# 数据加载与预处理
print("=== 数据加载与预处理 ===")

# 读取CSV数据
try:
    df_rent = pd.read_csv('张家口租房合并数据.csv')
    df_sale = pd.read_csv('张家口二手房合并数据.csv')
    print("✓ 成功读取CSV文件")
except FileNotFoundError:
    print("❌ 未找到CSV文件，请确保文件在当前目录")
    exit()

# 筛选需要保留的列
rent_columns_to_keep = ['区域', '面积', '租金(元/月)']
sale_columns_to_keep = ['区域', '建筑面积', '总价']

df_rent = df_rent[[col for col in rent_columns_to_keep if col in df_rent.columns]]
df_sale = df_sale[[col for col in sale_columns_to_keep if col in df_sale.columns]]

# 输出数据形状
print(f"租房数据形状: {df_rent.shape}")
print(f"二手房数据形状: {df_sale.shape}")

# 数据清洗
print("\n=== 数据清洗 ===")

# 提取数值特征并计算衍生指标
df_rent['面积_数值'] = df_rent['面积'].str.extract('(\d+\.?\d*)').astype(float)
df_rent['每平方米租金'] = df_rent['租金(元/月)'] / df_rent['面积_数值']

df_sale['建筑面积_数值'] = df_sale['建筑面积'].str.extract('(\d+\.?\d*)').astype(float)
df_sale['总价_数值'] = df_sale['总价'].str.extract('(\d+\.?\d*)').astype(float)
df_sale['每平方米房价'] = (df_sale['总价_数值'] * 10000) / df_sale['建筑面积_数值']

# 去除缺失值
df_rent = df_rent.dropna(subset=['面积_数值', '每平方米租金'])
df_sale = df_sale.dropna(subset=['建筑面积_数值', '每平方米房价'])

# 输出清洗后的数据量
print(f"清洗后租房数据: {df_rent.shape[0]} 条记录")
print(f"清洗后二手房数据: {df_sale.shape[0]} 条记录")

# 定义异常值处理函数（IQR方法）
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)  # 下四分位
    Q3 = df[col].quantile(0.75)  # 上四分位
    IQR = Q3 - Q1  # 四分位距
    lower_bound = Q1 - 1.5 * IQR  # 下界
    upper_bound = Q3 + 1.5 * IQR  # 上界
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)].copy()

# 处理异常值
df_sale_clean = remove_outliers(df_sale, '每平方米房价')
df_rent_clean = remove_outliers(df_rent, '每平方米租金')

# 输出异常值处理后的数据量
print(f"异常值处理后租房数据: {df_rent_clean.shape[0]} 条记录")
print(f"异常值处理后二手房数据: {df_sale_clean.shape[0]} 条记录")

# 构建模型
print("\n=== 构建模型 ===")

# 定义特征
categorical_feat = ['区域']  # 分类特征
numeric_feat_sale = ['建筑面积_数值']  # 二手房数值特征
numeric_feat_rent = ['面积_数值']  # 租房数值特征

# 房价预测原始模型（Model 1）
preprocessor_sale = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_feat),  # 分类特征独热编码
        ('num', 'passthrough', numeric_feat_sale)  # 数值特征直接传递
    ])

model_price = Pipeline(steps=[
    ('preprocessor', preprocessor_sale),  # 预处理
    ('regressor', LinearRegression())  # 线性回归模型
])

# 租金预测原始模型（Model 2）
preprocessor_rent = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_feat),
        ('num', 'passthrough', numeric_feat_rent)
    ])

model_rent = Pipeline(steps=[
    ('preprocessor', preprocessor_rent),
    ('regressor', LinearRegression())
])

# 房价预测增强模型（Model 1+）
preprocessor_sale_plus = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_feat),
        ('num', PolynomialFeatures(degree=2, include_bias=False), numeric_feat_sale)  # 生成二次项
    ])

model_price_plus = Pipeline(steps=[
    ('preprocessor', preprocessor_sale_plus),
    ('interaction', InteractionTransformer()),  # 添加交互特征
    ('regressor', LinearRegression())
])

# 租金预测增强模型（Model 2+）
preprocessor_rent_plus = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_feat),
        ('num', PolynomialFeatures(degree=2, include_bias=False), numeric_feat_rent)
    ])

model_rent_plus = Pipeline(steps=[
    ('preprocessor', preprocessor_rent_plus),
    ('interaction', InteractionTransformer()),
    ('regressor', LinearRegression())
])

# 模型训练与评估
print("\n=== 模型训练与评估 ===")

# 房价模型训练与评估
X_sale = df_sale_clean[categorical_feat + numeric_feat_sale]  # 特征
y_sale = df_sale_clean['每平方米房价']  # 目标变量

# 原始模型
model_price.fit(X_sale, y_sale)
y_sale_pred = model_price.predict(X_sale)
r2_price = r2_score(y_sale, y_sale_pred)  # R²评分
rmse_price = np.sqrt(mean_squared_error(y_sale, y_sale_pred))  # 均方根误差

# 增强模型
model_price_plus.fit(X_sale, y_sale)
y_sale_plus_pred = model_price_plus.predict(X_sale)
r2_price_plus = r2_score(y_sale, y_sale_plus_pred)
rmse_price_plus = np.sqrt(mean_squared_error(y_sale, y_sale_plus_pred))

# 租金模型训练与评估
X_rent = df_rent_clean[categorical_feat + numeric_feat_rent]
y_rent = df_rent_clean['每平方米租金']

# 原始模型
model_rent.fit(X_rent, y_rent)
y_rent_pred = model_rent.predict(X_rent)
r2_rent = r2_score(y_rent, y_rent_pred)
rmse_rent = np.sqrt(mean_squared_error(y_rent, y_rent_pred))

# 增强模型
model_rent_plus.fit(X_rent, y_rent)
y_rent_plus_pred = model_rent_plus.predict(X_rent)
r2_rent_plus = r2_score(y_rent, y_rent_plus_pred)
rmse_rent_plus = np.sqrt(mean_squared_error(y_rent, y_rent_plus_pred))

# 输出模型性能比较
print("\n【模型性能比较】")
print(f"Model 1（房价预测）- R²: {r2_price:.4f}, RMSE: {rmse_price:.2f}元/m²")
print(f"Model 1+（增强房价预测）- R²: {r2_price_plus:.4f}, RMSE: {rmse_price_plus:.2f}元/m²")
print(f"Model 2（租金预测）- R²: {r2_rent:.4f}, RMSE: {rmse_rent:.2f}元/m²/月")
print(f"Model 2+（增强租金预测）- R²: {r2_rent_plus:.4f}, RMSE: {rmse_rent_plus:.2f}元/m²/月\n")

# R²值差异分析
print("【R²值差异分析】")
if r2_price_plus > r2_price:
    print("Model 1+比Model 1的R²更高，因为添加了非线性特征和交互特征，能更好地捕捉数据中的复杂关系。")
else:
    print("Model 1比Model 1+的R²更高，可能是因为数据中非线性关系不明显，或存在过拟合。")

if r2_rent_plus > r2_rent:
    print("Model 2+比Model 2的R²更高，因为添加了非线性特征和交互特征，能更好地捕捉数据中的复杂关系。")
else:
    print("Model 2比Model 2+的R²更高，可能是因为数据中非线性关系不明显，或存在过拟合。")
print()

# 计算房价租金比
print("\n=== 计算房价租金比 ===")

# 生成增强模型的预测值
df_sale_clean['预测每平方米房价_增强'] = model_price_plus.predict(X_sale)
df_rent_clean['预测每平方米租金_增强'] = model_rent_plus.predict(X_rent)

# 计算各区域房价中位数
area_price_median_plus = df_sale_clean.groupby('区域')['预测每平方米房价_增强'].median().reset_index()
area_price_median_plus.columns = ['区域', '房价中位数（元/m²）']

# 计算各区域租金中位数（转换为年租金）
area_rent_median_plus = df_rent_clean.groupby('区域')['预测每平方米租金_增强'].median().reset_index()
area_rent_median_plus['年租金中位数（元/m²）'] = area_rent_median_plus['预测每平方米租金_增强'] * 12

# 合并数据并计算房价租金比
area_ratio_plus = pd.merge(area_price_median_plus, area_rent_median_plus, on='区域')
area_ratio_plus['房价租金比中位数'] = area_ratio_plus['房价中位数（元/m²）'] / area_ratio_plus['年租金中位数（元/m²）']

# 输出各区域房价租金比
print("【各区域房价租金比中位数（增强模型预测）】")
print(area_ratio_plus[['区域', '房价中位数（元/m²）', '年租金中位数（元/m²）', '房价租金比中位数']].round(2))

# 三种方法比较
print("\n=== 三种方法比较 ===")

# 方法1：基于实际值计算
actual_price_by_area = df_sale_clean.groupby('区域')['每平方米房价'].median().reset_index()
actual_rent_by_area = df_rent_clean.groupby('区域')['每平方米租金'].median().reset_index()
actual_rent_by_area['年租金'] = actual_rent_by_area['每平方米租金'] * 12
method1_ratio = pd.merge(actual_price_by_area, actual_rent_by_area, on='区域')
method1_ratio['房价租金比'] = method1_ratio['每平方米房价'] / method1_ratio['年租金']
method1_ratio = method1_ratio[['区域', '房价租金比']].rename(columns={'房价租金比': '方法1_房价租金比'})

# 方法2：基于原始模型预测值计算
df_sale_clean['预测每平方米房价_原始'] = model_price.predict(X_sale)
df_rent_clean['预测每平方米租金_原始'] = model_rent.predict(X_rent)

sale_pred_median = df_sale_clean.groupby('区域')['预测每平方米房价_原始'].median().reset_index()
rent_pred_median = df_rent_clean.groupby('区域')['预测每平方米租金_原始'].median().reset_index()
rent_pred_median['年租金'] = rent_pred_median['预测每平方米租金_原始'] * 12
method2_ratio = pd.merge(sale_pred_median, rent_pred_median, on='区域')
method2_ratio['房价租金比'] = method2_ratio['预测每平方米房价_原始'] / method2_ratio['年租金']
method2_ratio = method2_ratio[['区域', '房价租金比']].rename(columns={'房价租金比': '方法2_房价租金比'})

# 方法3：基于增强模型预测值计算
method3_ratio = area_ratio_plus[['区域', '房价租金比中位数']].rename(columns={'房价租金比中位数': '方法3_房价租金比'})

# 合并三种方法的结果并输出
all_ratios = pd.merge(pd.merge(method1_ratio, method2_ratio, on='区域'), method3_ratio, on='区域')
print("\n【三种方法的房价租金比比较】")
print(all_ratios.round(2))

# 样本量分析
print("\n【样本量分析】")
sample_size_by_area = df_sale_clean['区域'].value_counts().reset_index()
sample_size_by_area.columns = ['区域', '样本量']
print("各区域样本量：")
print(sample_size_by_area)

# 基于样本量评估结果可信度
if sample_size_by_area['样本量'].min() >= 5:
    print("由于各区域样本量都较为充足（≥5），三种方法的结果都有一定可信度，增强模型（方法3）可能更优。")
else:
    print("部分区域样本量较少，直接计算法（方法1）可能更可靠，因为模型在小样本上容易过拟合。")

# 生成可视化图表
print("\n=== 生成可视化图表 ===")

# 设置中文字体
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['figure.dpi'] = 300  # 提高分辨率
except:
    print("中文字体设置失败，将使用默认字体")

# 绘制增强模型的房价租金比柱状图
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(area_ratio_plus['区域'], area_ratio_plus['房价租金比中位数'], 
              color=['#4e79a7', '#f28e2b', '#e15759', '#76b7b2'])

# 添加数据标签
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}',
                xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom',
                fontsize=10, fontweight='bold')

# 设置图表属性
ax.set_xlabel('张家口区域', fontsize=12, fontweight='bold')
ax.set_ylabel('房价租金比中位数', fontsize=12, fontweight='bold')
ax.set_title('张家口各区域房价租金比中位数（增强模型预测）', fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='-')

# 调整y轴范围
ax.set_ylim(0, max(area_ratio_plus['房价租金比中位数']) * 1.2)

# 保存并显示图表
plt.tight_layout()
plt.savefig('figure_c.png', dpi=300, bbox_inches='tight')
plt.show()

# 绘制三种方法的房价租金比比较图
plt.figure(figsize=(12, 7))
bar_width = 0.25
index = np.arange(len(all_ratios['区域']))

# 绘制三组柱状图
plt.bar(index - bar_width, all_ratios['方法1_房价租金比'],
        bar_width, label='方法1: 实际值计算', color='#4e79a7')
plt.bar(index, all_ratios['方法2_房价租金比'],
        bar_width, label='方法2: 原始模型预测', color='#f28e2b')
plt.bar(index + bar_width, all_ratios['方法3_房价租金比'],
        bar_width, label='方法3: 增强模型预测', color='#e15759')

# 设置图表属性
plt.xlabel('张家口区域', fontsize=12, fontweight='bold')
plt.ylabel('房价租金比中位数', fontsize=12, fontweight='bold')
plt.title('三种方法计算的房价租金比比较', fontsize=14, fontweight='bold', pad=20)
plt.xticks(index, all_ratios['区域'])
plt.legend()
plt.grid(axis='y', alpha=0.3, linestyle='-')

# 保存并显示图表
plt.tight_layout()
plt.savefig('ratio_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# 分析完成提示
print("\n=== 分析完成 ===")
print("图表已保存为 'figure_c.png' 和 'ratio_comparison.png'")